# construction du modèle

## Importation des modules

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from joblib import dump
from utils import clean_text
from utils import evaluate_model
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

## Définition des fonctions

## Importation des données et brève exploration

In [2]:
data= pd.read_csv(r"QueryResults.csv")  
data.head()

,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount
0,Minification failed. Returning unminified cont...,<p>I have made my first website using MVC 5 wh...,|asp.net|asp.net-mvc|bundling-and-minification|,24395412,40,49177,0.0,12
1,Where should ajax request be made in Flux app?,<p>I'm creating a react.js application with fl...,|javascript|reactjs|reactjs-flux|,26632415,199,40698,0.0,6
2,Gem Install Debugger Error,<p>I am running Rails v4.0.2 on Mac OS 10.9.2 ...,|ruby-on-rails|ruby|rubygems|rvm|,24395453,25,28278,0.0,6
3,How to Change body parser limit in Firebase?,<p>I'm trying to proxy a file upload to fireba...,|node.js|express|firebase|google-cloud-functio...,45489810,7,946,NaN,1
4,iOS Android Material Design Hierarchical Timin...,<p>I want to do the animation introduced by An...,|ios|uicollectionview|calayer|material-design|,26632893,11,895,0.0,3


In [3]:
data.fillna('', inplace=True)

C:\Users\ouedr\AppData\Local\Temp\ipykernel_13472\3189924358.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.fillna('', inplace=True)


## Nettoyage

In [4]:
data['Title'] = data['Title'].apply(clean_text)
data['Body'] = data['Body'].apply(clean_text)

c:\Users\ouedr\Desktop\Machine_Learning_Engineer\Projet_Categorisez_automatiquement_des_questions\utils.py:18: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [5]:
tags, nested_tags = [], []
# recuperer top k des tags importants
for i in range(len(data)):
  tags.append(data["Tags"].iloc[i].split('|'))
for tag in tags:
  for subtag in tag:
    if subtag !="":
      nested_tags.append(subtag)

df_tags =  pd.DataFrame(nested_tags).rename(columns={0:"tags"})
df_tags = df_tags["tags"].value_counts().reset_index().sort_values("count", ascending=False)
real_tags = df_tags.head(30)["tags"].tolist()

In [6]:
# transformer le contenu de tag en list de simple tag
data['filtered_tags'] = data['Tags'].str.split("|")

In [7]:
# strip les virgules au debut des tags
def strip_and_filter_commas(tag_list):
    return [tag.strip(',') for tag in tag_list if tag.strip(',')]
data['filtered_tags'] = data['filtered_tags'].apply(strip_and_filter_commas)

In [8]:
# Vérifier si chaque ligne de 'filtered_tags' contient au moins un élément de 'real_tags'
data['contains_real_tag'] = data['filtered_tags'].apply(lambda x: any(tag in x for tag in real_tags))

In [9]:
# garder les tags pertinents et remplacer par others si impertinents
allowed_tags = real_tags

def filter_tags(tag_list):
    if any(tag in allowed_tags for tag in tag_list):
        return [tag for tag in tag_list if tag in allowed_tags]
    else:
        return ["Others"]   

	
data['filtered_tags'] = data['filtered_tags'].apply(filter_tags)

## Encodage

In [10]:
mlb = MultiLabelBinarizer()
mlb.fit(data['filtered_tags'])
y = mlb.transform(data['filtered_tags'])

## Vectorisation

In [12]:
X = data[['Title', 'Body']].apply(lambda x: ' '.join(x), axis=1)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

In [13]:
print(mlb.classes_)

['.net' 'Others' 'android' 'angular' 'angularjs' 'asp.net' 'c' 'c#' 'c++'
 'css' 'django' 'git' 'html' 'ios' 'java' 'javascript' 'jquery' 'linux'
 'mysql' 'node.js' 'objective-c' 'php' 'python' 'r' 'reactjs' 'ruby'
 'ruby-on-rails' 'sql' 'sql-server' 'swift' 'windows']


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Entrainement du modèle

In [15]:
model = LogisticRegression(random_state=42, max_iter = 2000)
pipeline = Pipeline([
      ('classifier', OneVsRestClassifier(model))
  ])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X)

## Evaluation du modèle

In [16]:
evaluate_model(pipeline, X_test, y_test, mlb)

Accuracy: 0.4024
Precision: 0.6588888192923241
Recall: 0.5279973095678493
F1-score: 0.5850032895366034


## Enregistrement du modèle et des fonctions

In [17]:
dump(pipeline, 'model.classifier')
dump(vectorizer, 'model_vectorizer')
dump(mlb, 'mlb_encoder')

['mlb_encoder']

## Test

In [18]:
# Exemple pour tester avec de nouvelles données

new_data = pd.DataFrame({
    'Title': ['django update model field based another field'],
    'Body': ['new django python want something used often java consider following model relevant class class item model model name model charfield model decimalfield default def self return self name class saledetail model model item model foreignkey item deposit model foreignkey deposit quantity model positiveintegerfield model decimalfield sale model foreignkey sale def self value self none self value want time item added saledetail update saledetail item saledetail new set used java pojos include logic setter method tried using python property encapsulate item property django update field directly hood would break automatic functionallity also tried subclassing foreignkey accept callback function find way call method container class want provide default want include logic view logic since conceptually think logic model server side use case would update total sale detail sale would like calculate user decides save sale save signal would work thanks']})

new_data['Title'] = new_data['Title'].apply(lambda x: clean_text(x))
new_data['Body'] = new_data['Body'].apply(lambda x: clean_text(x))

new_data_combined = new_data['Title'] + ' ' + new_data['Body']

new_data_combined = new_data[['Title', 'Body']].apply(lambda x: ' '.join(x), axis=1)
# Vectorisation des nouvelles données
new_X = vectorizer.transform(new_data_combined)

# Prédictions pour les nouvelles données
result = pipeline.predict(new_X)
# Transformation des prédictions en tags originaux
result = mlb.inverse_transform(result)
result

[('django', 'python')]

In [19]:
result

[('django', 'python')]